<a href="https://colab.research.google.com/github/omkarade/Neural-Machine-Translation-using-Encoder-Decoder-Model-English-to-Marathi/blob/main/Neural_Machine_Translation_using_Encoder_Decoder_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model


**Preprocessing:**

[data source](http://www.manythings.org/anki/ )

In [ ]:
lines= pd.read_table('/mar.txt', names=['eng','mar', 'del'])
lines=lines.drop(columns='del')
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.mar=lines.mar.apply(lambda x: x.lower())
# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.mar=lines.mar.apply(lambda x: re.sub("'", '', x))
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.mar=lines.mar.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.mar = lines.mar.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.mar=lines.mar.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.mar=lines.mar.apply(lambda x: re.sub(" +", " ", x))
# Add start and end tokens to target sequences
lines.mar = lines.mar.apply(lambda x : 'START_ '+ x + ' _END')


**Building Vocabulary**

In [ ]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of Marathi 
all_marathi_words=set()
for mar in lines.mar:
    for word in mar.split():
        if word not in all_marathi_words:
            all_marathi_words.add(word)



**calculating maximum length of source and target sequence ,building input_token and target_token.**

In [ ]:
# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

In [ ]:
# Max Length of target sequence
lenght_list=[]
for l in lines.mar:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar


37

In [ ]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_marathi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_marathi_words)
num_encoder_tokens, num_decoder_tokens


(5823, 14273)

In [ ]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens


14274

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())
lines = shuffle(lines)
lines.head(10)


,eng,mar
26637,the train finally arrived,START_ ट्रेन शेवटी पोहोचलीच _END
34353,everything tom does is illegal,START_ टॉम जे करतो ते सर्वकाही गैरकायदेशीर असत...
9586,wheres your baby,START_ तुझं बाळ कुठे आहे _END
15541,lets go and ask tom,START_ जाऊन टॉमला विचारूया _END
37050,that company produces microchips,START_ ती कंपनी मायक्रोचिप उत्पन्न करते _END
15877,they are vegetarians,START_ त्या शाकाहारी आहेत _END
37452,wine is poetry put into a bottle,START_ वाईन म्हणजे बाटलीत टाकलेलं काव्य _END
29560,he is staying with his aunt,START_ तो त्याच्या मामीबरोबर राहतोय _END
28284,it is fun to play baseball,START_ बेसबॉल खेळण्यात मजा येते _END
36860,ill go even if it rains heavily,START_ जोरजोरात पाऊस पडला तरीही मी जाईन _END


**train test Split**

In [ ]:
X, y = lines.eng, lines.mar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape


((41267,), (4586,))

**batch generator**

In [ ]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)


**Encoder - Decoder Model Architecture:**

In [ ]:
latent_dim = 50
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])


**Triaging model**

In [ ]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50


In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)


<ipython-input-35-b22c3ec5e69e>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


Epoch 1/50
322/322 [==============================] - 84s 217ms/step - loss: 0.9773 - acc: 0.1791 - val_loss: 0.8697 - val_acc: 0.2025
Epoch 2/50
322/322 [==============================] - 65s 203ms/step - loss: 0.8442 - acc: 0.2143 - val_loss: 0.8190 - val_acc: 0.2361
Epoch 3/50
322/322 [==============================] - 68s 211ms/step - loss: 0.7978 - acc: 0.2478 - val_loss: 0.7835 - val_acc: 0.2594
Epoch 4/50
322/322 [==============================] - 71s 221ms/step - loss: 0.7626 - acc: 0.2681 - val_loss: 0.7539 - val_acc: 0.2789
Epoch 5/50
322/322 [==============================] - 66s 206ms/step - loss: 0.7312 - acc: 0.2905 - val_loss: 0.7272 - val_acc: 0.3015
Epoch 6/50
322/322 [==============================] - 66s 206ms/step - loss: 0.7033 - acc: 0.3115 - val_loss: 0.7054 - val_acc: 0.3199
Epoch 7/50
322/322 [==============================] - 66s 206ms/step - loss: 0.6780 - acc: 0.3320 - val_loss: 0.6833 - val_acc: 0.3391
Epoch 8/50
322/322 [==============================] - 6

**Evaluation on Random data**

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [17]:
def Test(test,y):
  testt={'new':[test]}
  testt=pd.DataFrame(testt)
  y_orignal={'new':[y]}
  y_orignal=pd.DataFrame(y_orignal)
  testt=testt.new.tolist()
  y_orignal=y_orignal.new.tolist()
  train_gen = generate_batch(testt, y_orignal, batch_size = 1)
  (input_seq, actual_output), _ = next(train_gen)
  decoded_sentence = decode_sequence(input_seq)
  print('Predicted Marathi Translation:', decoded_sentence[:-4])
  print('Orignal Marathi Translation:', y)


In [18]:
Test(test='how are you',y='तू कसा आहेस')

1/1 [==============================] - 0s 19ms/step
Predicted Marathi Translation:  तू कसा आहेस 
Orignal Marathi Translation: तू कसा आहेस


In [19]:
Test(test='you are welcome',y='तुमचे स्वागत आहे')

1/1 [==============================] - 0s 22ms/step
Predicted Marathi Translation:  तू उंच आहेस का 
Orignal Marathi Translation: तुमचे स्वागत आहे


In [20]:
Test(test='can you do this',y='तुम्ही हे करू शकता का')

1/1 [==============================] - 0s 28ms/step
Predicted Marathi Translation:  हे तू करू शकता का 
Orignal Marathi Translation: तुम्ही हे करू शकता का


In [21]:
Test(test='do you do this',y='तुम्ही हे करता का')

1/1 [==============================] - 0s 21ms/step
Predicted Marathi Translation:  तू हे का 
Orignal Marathi Translation: तुम्ही हे करता का


In [22]:
Test(test='would you take tea',y='तू चहा घेशील का')

1/1 [==============================] - 0s 29ms/step
Predicted Marathi Translation:  तू चहा हवी का 
Orignal Marathi Translation: तू चहा घेशील का


In [23]:
Test(test='i saw you yesterday',y='मी तुला काल पाहिले')

1/1 [==============================] - 0s 23ms/step
Predicted Marathi Translation:  मी तुला काल पाहिलं 
Orignal Marathi Translation: मी तुला काल पाहिले
